# Cell types analysis

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7335069/

## Read in BMI data only for the left hemi (because AHBA)

In [28]:
import nibabel as nib
from neuromaps.datasets import available_annotations
from neuromaps import datasets
from neuromaps.resampling import resample_images
from neuromaps import parcellate
from nilearn.datasets import fetch_atlas_surf_destrieux
from neuromaps import datasets, images, nulls, resampling, plotting
from neuromaps.datasets import fetch_annotation
from neuromaps import transforms
from neuromaps.stats import compare_images
#from brainspace.plotting import plot_surf
import numpy as np
from nibabel import freesurfer

# Read the basic files

BMI_HCP=list(['/dagher/dagher11/filip/Obesity_maps/data/BMI_area_lh.gii'])

BMI_ctx=transforms.fsaverage_to_fsaverage(BMI_HCP, '164k', hemi='L')

# Read mask of medial wall for fsaverage

medwall_idx_fsaverage_lh=(freesurfer.io.read_label('/dagher/dagher10/yyau/toolbox/freesurfer/subjects/fsaverage/label/lh.Medial_wall.label',
                        read_scalars=False))


# Set all vertices of medial wall to nan

BMI_ctx[0].agg_data()[medwall_idx_fsaverage_lh] = np.nan
BMI=BMI_ctx[0].agg_data()

## Read in 308 parcels DK template annot file for lh (because AHBA has data for lh mostly)

In [29]:
DK308=freesurfer.io.read_annot('/dagher/dagher11/filip/Downloads/lh.500.aparc.annot')[0]

## Manually parcellate the data and remove 'unknown' and 'corpus callosum'

In [30]:
BMI_DK308=list()

for i in range(154):
    BMI_DK308.append(np.average(BMI[DK308==i]))
    
BMI_DK308.pop(0)
BMI_DK308.pop(7)

nan

## Create Euclidean distance matrix from coordinates

In [31]:
DK308_coords=np.loadtxt('/dagher/dagher11/filip/Downloads/308_regions_coordinates.txt')
DK152_coords=DK308_coords[0:152]

In [32]:
from scipy.spatial import distance

In [33]:
import itertools

DK152_euc=list()

for j,p in itertools.product(range(len(DK152_coords)), range(len(DK152_coords))):
    DK152_euc.append(distance.euclidean(DK152_coords[j],DK152_coords[p]))

DK152_euc=np.reshape(DK152_euc, (152, 152))

## Create BMI surrogate map

In [7]:
import pandas as pd
from brainsmash.mapgen.base import Base

brain_map_file = np.array(BMI_DK308)
dist_mat_file = DK152_euc
base = Base(x=brain_map_file, D=dist_mat_file)

surrogates = base(n=1000)
spin1000=pd.DataFrame(data=surrogates)
spin1000.to_csv("/dagher/dagher9/filip/BMI_152_1000spin_smash.csv",sep=',',index=False)

## Correlate

order: astro, endo, microglia, neuro-ex, neuro-in, oligo, OPC

In [34]:
Cells=np.loadtxt('/dagher/dagher11/filip/Downloads/cell_classes_AHBA_lh152parc.txt',skiprows=1)

In [35]:
for type in range(7):
        main_corr=np.corrcoef(BMI_DK308, Cells.T[type])[1,0]
        print(type)
        print(main_corr)
        corrs=list()
        for spin in range(1000):
            corrs.append(np.corrcoef(surrogates[spin], Cells.T[type])[1,0])
        print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)
        

0
0.2724063575191533
0.08691308691308691
1
-0.03177640991105796
0.8691308691308691
2
-0.09217125621097658
0.18281718281718282
3
-0.11580209623838199
0.26973026973026976
4
-0.07924726056841148
0.35664335664335667
5
0.055449095245651586
0.5834165834165834
6
0.23582189962455172
0.28771228771228774


# Gene analysis

## Get genetics data

In [24]:
import abagen
atlas = abagen.fetch_desikan_killiany()

In [36]:
files = abagen.fetch_microarray(data_dir='/dagher/dagher9/filip/', donors='all', verbose=0)

In [37]:
expression_125 = abagen.get_expression_data('/home/bic/fmorys/nnt-data/atl-cammoun2012/MNI152NLin2009aSym/atl-Cammoun2012_space-MNI152NLin2009aSym_res-125_deterministic.nii.gz',
                                            '/dagher/dagher11/filip/Obesity_maps/data/Cammoun_125_all.csv', 
                                            data_dir='dagher/dagher9/filip/',
                                            missing='interpolate')

/dagher/dagher12/filip/software/conda/envs/R/lib/python3.9/site-packages/abagen/probes_.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  reannotated.loc[:, 'entrez_id'] = reannotated['entrez_id'].astype('int')
/dagher/dagher12/filip/software/conda/envs/R/lib/python3.9/site-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/dagher/dagher12/filip/software/conda/envs/R/lib/python3.9/site-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
 

In [38]:
expression_125

gene_symbol,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
label,,,,,,,,,,,,,,,,,,,,,
1,0.622944,0.690267,0.439806,0.393562,0.428239,0.402672,0.494042,0.859539,0.407667,0.639381,...,0.485621,0.617660,0.632557,0.398216,0.464592,0.496432,0.758898,0.600131,0.390074,0.473967
2,0.585196,0.640841,0.432136,0.426100,0.536842,0.490134,0.480824,0.798251,0.462878,0.637841,...,0.442843,0.646946,0.459889,0.355813,0.494127,0.473319,0.719956,0.671837,0.343565,0.569040
3,0.495600,0.661128,0.458444,0.382050,0.552058,0.623924,0.459618,0.859767,0.485064,0.593882,...,0.499877,0.612031,0.455092,0.403025,0.604985,0.432173,0.756675,0.634607,0.327567,0.438100
4,0.737949,0.710726,0.325531,0.282371,0.373976,0.529114,0.428175,0.831062,0.563569,0.558076,...,0.387572,0.742470,0.667261,0.594402,0.497352,0.130365,0.621134,0.570750,0.301022,0.652820
5,0.511351,0.647186,0.403263,0.427298,0.508712,0.513535,0.362262,0.814090,0.548853,0.510701,...,0.542103,0.641034,0.455598,0.456249,0.549933,0.474662,0.749055,0.587563,0.293146,0.554342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0.675548,0.243978,0.599180,0.534929,0.461017,0.497506,0.549787,0.164591,0.535338,0.511910,...,0.414569,0.340405,0.313896,0.710642,0.526642,0.752233,0.246004,0.256645,0.591600,0.805949
231,0.551010,0.285390,0.522781,0.656677,0.535230,0.472690,0.318485,0.183048,0.600951,0.423426,...,0.287061,0.474273,0.255185,0.883280,0.657603,0.606432,0.330304,0.133929,0.409622,0.726780
232,0.484392,0.337209,0.420463,0.592364,0.582342,0.593309,0.283602,0.512382,0.651044,0.194806,...,0.335131,0.541032,0.727145,0.696629,0.453882,0.572417,0.327288,0.261803,0.591652,0.473318


## Use Cammoun 125 parcellation for BMI data

In [40]:
from netneurotools import datasets as nntdatasets
from neuromaps import datasets as nmdatasets
from neuromaps.parcellate import Parcellater
import pandas as pd

cammoun = nntdatasets.fetch_cammoun2012(version="MNI152NLin2009aSym")
info = pd.read_csv(cammoun['info'])
labels = info.query('scale=="scale125"')['label']  

parc = Parcellater(cammoun['scale125'], 'mni152', hemi='L')  
mymap_parc = parc.fit_transform('/dagher/dagher9/filip/DBM/HCP_DBM_Slvl/beta_0003.nii', 'mni152')

## Add labels to expression data

In [41]:
expression1=expression_125
expression1['struct']=labels.values
expression1=expression1.reset_index()
expression1=expression1.set_index('struct')
expression1=expression1[115:234]

In [42]:
expression1.to_csv('/dagher/dagher11/filip/Obesity_maps/data/abagenes_Cammoun125.csv', sep='\t')

## Get surrogate maps for BMI only L hemisphere

In [77]:
Euc_125_L=pd.read_csv('/dagher/dagher11/filip/Downloads/Euc_dist125_L_brainstem.txt', sep=',',header=None)

In [79]:
import pandas as pd
from brainsmash.mapgen.base import Base

brain_map_file = mymap_parc[0,115:234]
dist_mat_file = Euc_125_L.values
base = Base(x=brain_map_file, D=dist_mat_file)

surrogates = base(n=10000)
spin1000=pd.DataFrame(data=surrogates)
spin1000.to_csv("/dagher/dagher9/filip/BMI_abagen_L_Cammoun125_10000spin_smash.csv",sep=',',index=False)

## Correlate

In [222]:
gene_corrs=pd.DataFrame([[0, 0, 0]], columns=['gene','corrcoef','p-val'], index=['gene'])
index=0

for gene in expression1:
        main_corr=np.corrcoef(mymap_parc[0,115:234], expression1[gene])[1,0]
        print('Gene number '+str(index))
        index=index+1
        #print(main_corr)
        corrs=list()
        for spin in range(10000):
            corrs.append(np.corrcoef(surrogates[spin], expression1[gene])[1,0])
            p=(sum(i > abs(main_corr) for i in np.absolute(corrs))/10001)
        gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'], 
                                                  index=[gene]))
        
gene_corrs.to_csv(path_or_buf='/dagher/dagher11/filip/Obesity_maps/data/gene_maps_correlations_10000.csv', sep=',',
                 header=True)

Gene number 0
Gene number 1
Gene number 2
Gene number 3
Gene number 4
Gene number 5
Gene number 6
Gene number 7
Gene number 8
Gene number 9
Gene number 10
Gene number 11
Gene number 12
Gene number 13
Gene number 14
Gene number 15
Gene number 16
Gene number 17
Gene number 18
Gene number 19
Gene number 20
Gene number 21
Gene number 22
Gene number 23
Gene number 24
Gene number 25
Gene number 26
Gene number 27
Gene number 28
Gene number 29
Gene number 30
Gene number 31
Gene number 32
Gene number 33
Gene number 34
Gene number 35
Gene number 36
Gene number 37
Gene number 38
Gene number 39
Gene number 40
Gene number 41
Gene number 42
Gene number 43
Gene number 44
Gene number 45
Gene number 46
Gene number 47
Gene number 48
Gene number 49
Gene number 50
Gene number 51
Gene number 52
Gene number 53
Gene number 54
Gene number 55
Gene number 56
Gene number 57
Gene number 58
Gene number 59
Gene number 60
Gene number 61
Gene number 62
Gene number 63
Gene number 64
Gene number 65
Gene number 66
Gene 

Gene number 519
Gene number 520
Gene number 521
Gene number 522
Gene number 523
Gene number 524
Gene number 525
Gene number 526
Gene number 527
Gene number 528
Gene number 529
Gene number 530
Gene number 531
Gene number 532
Gene number 533
Gene number 534
Gene number 535
Gene number 536
Gene number 537
Gene number 538
Gene number 539
Gene number 540
Gene number 541
Gene number 542
Gene number 543
Gene number 544
Gene number 545
Gene number 546
Gene number 547
Gene number 548
Gene number 549
Gene number 550
Gene number 551
Gene number 552
Gene number 553
Gene number 554
Gene number 555
Gene number 556
Gene number 557
Gene number 558
Gene number 559
Gene number 560
Gene number 561
Gene number 562
Gene number 563
Gene number 564
Gene number 565
Gene number 566
Gene number 567
Gene number 568
Gene number 569
Gene number 570
Gene number 571
Gene number 572
Gene number 573
Gene number 574
Gene number 575
Gene number 576
Gene number 577
Gene number 578
Gene number 579
Gene number 580
Gene num

KeyboardInterrupt: 

In [212]:
from statsmodels.stats import multitest

corrected=multitest.multipletests(gene_corrs['p-val'], alpha=0.05, method='fdr_bh')

In [213]:
gene_corrs['cor_p']=corrected[1]

In [220]:
for gene in (gene_corrs[gene_corrs['p-val']<0.05].index.values):
    print(gene)

gene
ABCF3
ABCG1
ABHD16A
ABHD18
ABRAXAS2
ACADSB
ACAP2
ACAP3
ACAT2
ACKR4
ACSF3
ACSM1
ACTN2
ADA2
ADAMTSL2
ADCY8
ADCYAP1
ADGRG5
ADPRHL1
ADRA1B
ADRM1
ADTRP
AGBL3
AGFG1
AGL
AGTPBP1
AKAP11
AKAP12
AKAP8
ALAS1
ALDH1L2
ALDH5A1
ALDOA
ALKBH3
ALKBH5
ALMS1
ALMS1P1
ALPP
ALS2CL
ALS2CR12
ANKFY1
ANKRA2
ANKRD17
ANKRD27
ANKRD28
ANKRD30B
ANKRD36B
ANKRD36BP2
ANKRD44
ANKRD45
ANKS1B
ANP32D
AP1M1
AP2S1
AP4B1
AP4M1
APCDD1L
APEH
APLP2
APOL5
AQP2
ARFIP2
ARHGAP20
ARHGAP25
ARHGDIG
ARHGEF33
ARID3B
ARL4D
ARL5A
ARL8B
ARSK
ARTN
ASB16
ASB18
ASB2
ASB6
ASGR2
ASNSP1
ATAD2B
ATE1
ATG101
ATG2A
ATG5
ATG9A
ATP10D
ATP13A2
ATP2C1
ATP5PF
ATP6V0A1
ATP6V1A
ATRX
ATXN3
AURKA
AURKAIP1
AURKC
AUTS2
AVP
AZIN2
BAALC
BAIAP2L2
BANP
BCAS3
BCL2L11
BCL2L13
BEND5
BEX4
BHLHE22
BIN3-IT1
BIRC2
BNIP1
BNIP3
BOD1
BRD2
BTBD1
BTBD10
BTF3
BTG1
BTN2A3P
BTRC
C11orf49
C11orf74
C12orf29
C14orf28
C19orf44
C20orf144
C20orf96
C22orf15
C2orf49
C2orf70
C3orf38
C5orf22
C8orf46
C9orf139
CAAP1
CABLES1
CACNA1E
CACNB2
CALHM1
CALM3
CALN1
CAMK1G
CAPNS1
CAPRIN2
CARF
CAS

# Take genes significantly related to BMI according to 2018 GWAS and get an average map of distribution in the brain

## From Yengo

In [315]:
import itertools
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

Yengo = pd.read_csv('/dagher/dagher11/filip/Obesity_maps//data/Yengo_genes.csv')

common_genes=intersection(list(expression1.columns), list(itertools.chain(*Yengo.values.tolist())))

In [316]:
gene_maps_BMI=expression1[common_genes]
gene_maps_BMI['mean']=gene_maps_BMI.mean(axis=1)

/tmp/ipykernel_5401/1895017346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_maps_BMI['mean']=gene_maps_BMI.mean(axis=1)


### Correlate BMI map with gene map

In [317]:
main_corr=np.corrcoef(mymap_parc[0,115:234], gene_maps_BMI['mean'])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], gene_maps_BMI['mean'])[1,0])
p=(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)
print(p)

0.07943551323107982
0.7022977022977023


## Plot genes on surface

## Reverse parcellate data - remove unknown + CC

In [471]:
from nibabel import freesurfer

img_annot = freesurfer.read_annot('/home/bic/fmorys/nnt-data/atl-cammoun2012/fsaverage/atl-Cammoun2012_space-fsaverage_res-125_hemi-L_deterministic.annot')
BMI_genes_voxelwise=img_annot[0]

In [472]:
BMI_mean_genes=gene_maps_BMI['mean'][0:111]

In [473]:
dictionary_labels=pd.DataFrame(img_annot[2], np.unique(img_annot[0]))

In [474]:
dictionary_labels.index.name = 'label'
dictionary_labels=dictionary_labels.drop(0)
dictionary_labels=dictionary_labels.drop(4)

In [475]:
BMI_label=pd.DataFrame(BMI_mean_genes.values, dictionary_labels.index)

In [476]:
for i in range(1,113):
    if i==4:
        BMI_genes_voxelwise[np.where(BMI_genes_voxelwise == i)]=0
        continue
    BMI_genes_voxelwise[np.where(BMI_genes_voxelwise == i)]=BMI_label.loc[i]*100000

In [477]:
np.savetxt('/dagher/dagher11/filip/Obesity_maps/data/GWAS_genes_map_Yengo.csv', BMI_genes_voxelwise, delimiter=',')

## For volumetric atlas

In [291]:
import nibabel as nib

img = nib.load(cammoun['scale125'])
BMI_genes_parcel=img

In [293]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(gene_maps_BMI['mean'])

struct
lateralorbitofrontal_1        0.515078
lateralorbitofrontal_2        0.521666
lateralorbitofrontal_3        0.530486
lateralorbitofrontal_4        0.525705
parsorbitalis_1               0.519592
frontalpole_1                 0.516944
medialorbitofrontal_1         0.528438
medialorbitofrontal_2         0.518535
parstriangularis_1            0.534659
parsopercularis_1             0.529556
parsopercularis_2             0.532756
rostralmiddlefrontal_1        0.533203
rostralmiddlefrontal_2        0.517177
rostralmiddlefrontal_3        0.527652
rostralmiddlefrontal_4        0.525446
rostralmiddlefrontal_5        0.504273
rostralmiddlefrontal_6        0.527186
superiorfrontal_1             0.520097
superiorfrontal_2             0.526609
superiorfrontal_3             0.528011
superiorfrontal_4             0.543615
superiorfrontal_5             0.516398
superiorfrontal_6             0.521448
superiorfrontal_7             0.515942
superiorfrontal_8             0.515705
superiorfrontal_9 

In [306]:
labels.index = range(1,235)
labels_BMI=labels[115:len(labels)]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(labels_BMI)

116        lateralorbitofrontal_1
117        lateralorbitofrontal_2
118        lateralorbitofrontal_3
119        lateralorbitofrontal_4
120               parsorbitalis_1
121                 frontalpole_1
122         medialorbitofrontal_1
123         medialorbitofrontal_2
124            parstriangularis_1
125             parsopercularis_1
126             parsopercularis_2
127        rostralmiddlefrontal_1
128        rostralmiddlefrontal_2
129        rostralmiddlefrontal_3
130        rostralmiddlefrontal_4
131        rostralmiddlefrontal_5
132        rostralmiddlefrontal_6
133             superiorfrontal_1
134             superiorfrontal_2
135             superiorfrontal_3
136             superiorfrontal_4
137             superiorfrontal_5
138             superiorfrontal_6
139             superiorfrontal_7
140             superiorfrontal_8
141             superiorfrontal_9
142         caudalmiddlefrontal_1
143         caudalmiddlefrontal_2
144         caudalmiddlefrontal_3
145           

/tmp/ipykernel_5401/1691492260.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  labels_BMI=labels[115:len(labels)]


In [308]:
BMI_mean_genes.index = range(116,235)

In [310]:
data=BMI_genes_parcel.get_fdata()

data[np.where(data < 116)]=0

for i in range(116,235):
    data[np.where(data == i)]=BMI_mean_genes[i]   
    
BMI_genes_img = nib.Nifti1Image(data, BMI_genes_parcel.affine, BMI_genes_parcel.header)
nib.save(BMI_genes_img, '/home/bic/fmorys/Desktop/image.nii')

# OLD

### Abagen order only LH

In [342]:
struct_aba=np.array(['bankssts',
'caudalanteriorcingulate',
'caudalmiddlefrontal',
'cuneus',
'entorhinal',
'fusiform',
'inferiorparietal',
'inferiortemporal',
'isthmuscingulate',
'lateraloccipital',
'lateralorbitofrontal',
'lingual',
'medialorbitofrontal',
'middletemporal',
'parahippocampal',
'paracentral',
'parsopercularis',
'parsorbitalis',
'parstriangularis',
'pericalcarine',
'postcentral',
'posteriorcingulate',
'precentral',
'precuneus',
'rostralanteriorcingulate',
'rostralmiddlefrontal',
'superiorfrontal',
'superiorparietal',
'superiortemporal',
'supramarginal',
'frontalpole',
'temporalpole',
'transversetemporal',
'insula',
'thalamusproper',
'caudate',
'putamen',
'pallidum',
'accumbensarea',
'hippocampus',
'amygdala',
'bankssts',
'caudalanteriorcingulate',
'caudalmiddlefrontal',
'cuneus',
'entorhinal',
'fusiform',
'inferiorparietal',
'inferiortemporal',
'isthmuscingulate',
'lateraloccipital',
'lateralorbitofrontal',
'lingual',
'medialorbitofrontal',
'middletemporal',
'parahippocampal',
'paracentral',
'parsopercularis',
'parsorbitalis',
'parstriangularis',
'pericalcarine',
'postcentral',
'posteriorcingulate',
'precentral',
'precuneus',
'rostralanteriorcingulate',
'rostralmiddlefrontal',
'superiorfrontal',
'superiorparietal',
'superiortemporal',
'supramarginal',
'frontalpole',
'temporalpole',
'transversetemporal',
'insula',
'thalamusproper',
'caudate',
'putamen',
'pallidum',
'accumbensarea',
'hippocampus',
'amygdala',
'brainstem'])


In [10]:
values = np.random.rand(219)
scale = 'scale125'
cammoun = fetch_cammoun2012('fsaverage', verbose=False)[scale]
plot_fsaverage(values, order='LR',lhannot=cammoun.lh,rhannot=cammoun.rh)# doctest: +SKIP

ImportError: Cannot use plot_fsaverage() if pysurfer is not installed. Please install pysurfer and try again.

In [258]:
parcellated_map=[val for sublist in np.ndarray.tolist(mymap_parc) for val in sublist][0:42]
parc_dict=dict(zip(list(labels), parcellated_map))

In [12]:
pip install mayavi

  Using cached mayavi-4.7.4.tar.gz (7.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [821 lines of output]
      running bdist_wheel
      running build
      ----------------------------------------------------------------------
      Building TVTK classes... vtkContextDevice2D: Ignoring method: Get/SetViewportRect
      default: [-1318507920, 32750, -1318507920, 32750], range: None
      vtkContextDevice2D: Ignoring method: Get/SetViewportSize
      default: [-1318507920, 32750], range: None
      vtkEventDataForDevice: Ignoring method: Get/SetAction
      default: -1, range: None
      vtkEventDataForDevice: Ignoring method: Get/SetDevice
      default: -1, range: None
      vtkFXAAOptions: Ignoring method: Get/SetDebugOptionValue
      default: 0, range: None
      vtkIncrementalForceLayout: Ignoring method: Get/SetGravityPoint
      default: [200.0, 200.0], ran

Failed to build mayavi
  Running setup.py install for mayavi ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for mayavi did not run successfully.
  │ exit code: 1
  ╰─> [846 lines of output]
      running install
      running build
      ----------------------------------------------------------------------
      Deleting possibly old TVTK classes
      Building TVTK classes... vtkContextDevice2D: Ignoring method: Get/SetViewportRect
      default: [-1973625808, 32739, -1973625808, 32739], range: None
      vtkContextDevice2D: Ignoring method: Get/SetViewportSize
      default: [-1973625808, 32739], range: None
      vtkEventDataForDevice: Ignoring method: Get/SetAction
      default: -1, range: None
      vtkEventDataForDevice: Ignoring method: Get/SetDevice
      default: -1, range: None
      vtkFXAAOptions: Ignoring method: Get/SetDebugOptionValue
      default: 0, range: None
      vtkIncrementalForceLayout: Ignoring method: Get/SetGravityPoint
   

Note: you may need to restart the kernel to use updated packages.


In [260]:
reordered_parc_dict = {k: parc_dict[k] for k in order}